In [14]:
import cv2
import numpy as np
import os
from PIL import Image

In [23]:
    
    path = cv2.data.haarcascades + 'haarcascade_frontalface_default.xml'
    print("Path to cascade:", path)
    print("Exists:", os.path.exists(path))


Path to cascade: C:\Users\Poorni\AppData\Roaming\Python\Python312\site-packages\cv2\data\haarcascade_frontalface_default.xml
Exists: True


In [28]:
def create_user(face_id,name):
    web=cv2.VideoCapture(0)
    web.set(3,640)
    web.set(4,480)
    
    face = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    
    if face.empty():
        print("Failed to load Haar Cascade from:", cascade_path)
    web.release()
    return
    
    f_dir='data'
    f_name='dst'
    path=os.path.join(f_dir,f_name)
    if not os.path.exists(f_dir):
        os.mkdir(f_dir)
    if not os.path.isdir(path):
        os.mkdir(path)

    counter=0
    while(True):
        ret,img=web.read()
        img=cv2.flip(img,1)
        gray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
        multi_face=face.detectMultiScale(gray,1.2,5)
        for (x,y,w,h) in multi_face:
            cv2.rectangle(img, (x,y), (x+w,y+h), (0,0,255),4)
            counter+=1

            file_path = f"{path}/{name}.{face_id}.{counter}.jpg"
            cv2.imwrite(file_path, gray[y:y + h, x:x + w])

            cv2.imshow("Capturing Face", img)
        web.release()


In [29]:
create_user(1,"Poorni")

In [7]:
!pip install opencv-contrib-python

In [21]:
def train():
    db = 'data/dst'  # Full path to where your captured images are stored
    recognizer = cv2.face.LBPHFaceRecognizer_create()
    detector = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

    faceSamples = []
    ids = []

    # Loop through subfolders in dst (like data/dst/Poorvi)
    for root, dirs, files in os.walk(db):
        for file in files:
            if file.endswith(".jpg") or file.endswith(".png"):
                imagePath = os.path.join(root, file)
                PIL_img = Image.open(imagePath).convert('L')  # convert to grayscale
                img_numpy = np.array(PIL_img, 'uint8')

                try:
                    id = int(os.path.split(imagePath)[-1].split('.')[1])
                except:
                    print(f"Skipping file with invalid name format: {imagePath}")
                    continue

                faces = detector.detectMultiScale(img_numpy)

                for (x, y, w, h) in faces:
                    faceSamples.append(img_numpy[y:y + h, x:x + w])
                    ids.append(id)

    if len(faceSamples) == 0:
        print(" No faces found. Make sure images exist in 'data/dst' and are named properly.")
        return 0

    recognizer.train(faceSamples, np.array(ids))
    recognizer.write('trainer.yml')
    print('\n [INFO] {0} unique face(s) trained. Exiting program.'.format(len(np.unique(ids))))
    return len(np.unique(ids))

In [22]:
train()

 No faces found. Make sure images exist in 'data/dst' and are named properly.


0

In [ ]:
def recognize(names):
    recognizer = cv2.face.LBPHFaceRecognizer_create()
    recognizer.read('trainer.yml')

    cascadePath = "haarcascade_frontalface_default.xml"
    faceCascade = cv2.CascadeClassifier(cascadePath)

    font = cv2.FONT_HERSHEY_SIMPLEX
    id = 0
    name = ""
    face_count = 0

    cam = cv2.VideoCapture(0)
    cam.set(3,640)
    cam.set(4,480)

    minW = 0.1*cam.get(3)
    minH = 0.1*cam.get(4)
    while True:
        ret, img = cam.read()
        img = cv2.flip(img, 1)
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

        faces = faceCascade.detectMultiScale(
                gray,
                scaleFactor = 1.2,
                minNeigh = 5,
                minSize = (int(minW), int(minH)),
            )

        for (x,y,w,h) in faces:
            cv2.rectangle(img, (x,y), (x+w,y+h), (0,255,0), 2)

            id, confidence = recognizer.predict(gray[y:y+h, x:x+w])
            if(confidence<70):
                id = names[id]
            else:
                id = 'unknown'
                roll = None
                confidence = " {0}%".format(round(100 - confidence))
                text = "I can't recognize You"

            if  name == id:
                face_count += 1
                if face_count > 21:
                    fount_count = -100
            else:
                name = id
                face_count = 0

            cv2.putText(img, str(id), (x + 5, y - 5), font, 1, (255, 255, 255), 2)
            cv2.putText(img, str(confidence), (x + 5, y + h - 5), font, 1, (255, 255, 0), 1)

        cv2.imshow("Camera", img)